# Introduction

This notebook is the second in the Makemore series. In part 1 we implemented Makemore using a bigram model. We looked at ONE previous character and produced a probability distribution of what character is likely to come next. We did that using two different approaches and reached the same results. First, we used counts and normalized them into probabilities. Second, we used a simple neural network (i.e. linear layer) to produce those same probabilities.

The limit of our implementation in part 1 is that it only looks at ONE previous character; and because the bigram model took only ONE character as context, the prediction of the bigram were **not** good. If we were to use consider MORE than one character when predicting the next one, the approach we used in part-1 (storing everything in a tensor) becomes unsustainable because _the number of combinations we will have to store in a tensor grows exponentially with the amount characters we take as context_. For instance, if we were to take just three characters as context when making a prediction, we would have to store $27 \times 27 \times 27 = 19683$ possibilities in a tensor. That's way to many possibilities, the majority of these possibilities will have very few counts, Andrej said.

That's why for this second part, we are moving away from the Bigram model. This time, we are implementing a Multi-Layer Percepton to predict the next character. The modeling approach we are going to adopt follows the paper: [Bengio et al. 2003](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf) 

# Bengio et al. 2003 (MLP language model) paper walkthrough

This paper, Andrej said, is not the first to have proposed the use of MLPs to predict the next character/token in a sequence, but was very influencial and is very often cited. Since the paper is long ($19$ pages) Andrej decided to give us gist of it, but invited us to read the entire work. This paper is what we are going to implement in this notebook.

# Re-building our training dataset

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # For making figures
%matplotlib inline

In [2]:
# Read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

Now we build the vocabulary of characters and mapping from/to integers. It is important to recall that we are building a character-level language model, unlike the paper who is describing a word-level language model.

In [4]:
# Build the vocabulary of characters and mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i + 1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


We now want to set up the dataset such that we can feed training examples to the neural network easily. We are going to refurbish code we wrote in the previous tutorial. To form the dataset in the first part, we added the first part of the bigram (the context) in a tensor, `xs`... and the second part (the correct prediction) in another tensor, `ys`. 

We are doing something very similar here, but this time Andrej is adding MORE than ONE character as context. `X` will store the input of the neural network and `Y` will store the correct labels. If, out of curiosity, you uncomment the commented lines, the code print all the training examples _per_ word in our dataset. For the output to be manageable, I suggest doing it for a couple words and not the entire dataset 😉.

In [5]:
block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    idx = stoi[ch]
    X.append(context)
    Y.append(idx)
    #print(''.join(itos[i] for i in context), '--->', itos[idx])
    context = context[1:] + [idx] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

The shape of the dataset is as follows

In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

Our training set `X` contains $32$ training examples of $3$ characters (i.e. context) each when **considering only $5$ words**. If we were to consider all of the words, our training set would contain $228146$ training examples, each with a context size of $3$. Also, printing the `dtype` indicates that tensors `X` and `Y` are storing `int64` values. It is because we are not storing the characters directly but rather the unique integers we assigned to each of them using `stoi` defined earlier in this notebook.

Feel free to print out `X` or `Y` and see what's inside.

In [7]:
#X

In [8]:
#Y

# Implementing the embedding lookup table

We have 27 possible characters and we are going to embed each of them in a lower dimensional space. In the paper, they have $17,000$ total words and embed them all into a 30 dimensional space, which Andrej said was small. Since we have just 27 characters like mentioned earlier, Andrej suggested we start with 2D embedding space. That means, each of the 27 letters will be associated with a 2D embedding vector. As a result, our embedding matrix will be of the shape $(27 \times 2)$. In that sense, it is reasonable to look at the embedding matrix as a **lookup table**.

_REMEMBER: Those embedding vectors are (initially) randomly generated._

In [9]:
C = torch.randn((27, 2))

In [10]:
C

tensor([[-1.0041, -1.3760],
        [ 0.1064,  0.3253],
        [ 0.1916,  0.0938],
        [-0.5132,  1.9619],
        [-0.6944,  0.5034],
        [-1.8877, -0.5583],
        [ 0.8845,  0.3055],
        [-0.4529, -1.1617],
        [ 1.6599, -1.1366],
        [-1.6619, -0.0768],
        [-1.5524,  1.3347],
        [ 1.1862, -0.9254],
        [ 1.8109, -0.4273],
        [ 1.2296, -0.3806],
        [-0.0960, -0.3728],
        [ 1.2858,  0.7160],
        [ 0.1043,  1.8602],
        [ 1.1328, -1.5410],
        [ 0.2834,  1.8160],
        [ 0.8093, -0.8099],
        [-0.8473, -0.9647],
        [-0.0997, -0.7861],
        [-1.3248,  0.2001],
        [-1.7273, -0.0787],
        [-0.6035, -0.7751],
        [ 2.9611, -0.2968],
        [-1.3237, -0.7269]])

There are multiple ways to index into the embedding matrix `C`. We can do it by using directly using the row index, or more interestingly, we can select specific rows of the matrix by multiplying it with a one-hot encoded vector. We illustrated this in "NOTE 1" section of the `part-1` notebook. For simplicity, Andrej decided to use numbers for indexing. With Pytorch, it is possible to do single-dimension indexing, meaning use a single number to select a row in a tensor or use a 1-D tensor to select multiple rows at once. Andrej showed us that is also possible to select rows in a tensor using a 2-D tensor.

Using 2-D indexing, we can simultaneously retreive the embedding vector of all the integers in the training set `X` like so:

In [11]:
C[X]

tensor([[[-1.0041, -1.3760],
         [-1.0041, -1.3760],
         [-1.0041, -1.3760]],

        [[-1.0041, -1.3760],
         [-1.0041, -1.3760],
         [-1.8877, -0.5583]],

        [[-1.0041, -1.3760],
         [-1.8877, -0.5583],
         [ 1.2296, -0.3806]],

        [[-1.8877, -0.5583],
         [ 1.2296, -0.3806],
         [ 1.2296, -0.3806]],

        [[ 1.2296, -0.3806],
         [ 1.2296, -0.3806],
         [ 0.1064,  0.3253]],

        [[-1.0041, -1.3760],
         [-1.0041, -1.3760],
         [-1.0041, -1.3760]],

        [[-1.0041, -1.3760],
         [-1.0041, -1.3760],
         [ 1.2858,  0.7160]],

        [[-1.0041, -1.3760],
         [ 1.2858,  0.7160],
         [ 1.8109, -0.4273]],

        [[ 1.2858,  0.7160],
         [ 1.8109, -0.4273],
         [-1.6619, -0.0768]],

        [[ 1.8109, -0.4273],
         [-1.6619, -0.0768],
         [-1.3248,  0.2001]],

        [[-1.6619, -0.0768],
         [-1.3248,  0.2001],
         [-1.6619, -0.0768]],

        [[-1.3248,  0

In [12]:
C[X].shape

torch.Size([32, 3, 2])

Why do we get a 3D matrix back? Remember each character is represented by a number in tensor `X`. For each number in `X` the corresponding embedding is returned. Since we have three numbers per row in `X`, the  is printed as a _series of $(3 \times 2)$ matrices_. Makes sense? That's my way of looking at it.

Andrej for instance said, for each of the elements in the $(32 \times 3)$ matrix (i.e. the training set `X`), we retreived the corresponding the embedding. Makes sense?

Besides, the following drawing helped solidify my understanding:

[IMAGE HERE]

With the embedding of all the integers in `X` selected, Andrej stored them in the `embed` variable like so:

In [13]:
embed = C[X]
embed.shape

torch.Size([32, 3, 2])

# Implementing the hidden layer + internals of torch. Tensor: storage, views

In this section of the video, we implement the hidden layer of the of the neural network. Andrej decided to call the weights of this hidden layer `W1`, so we are going to do just like him.

What should be the shape of `W1`? It depends on two things. (1) the number of neurons in the hidden layer. This number determines the number columns in `W1`. (2) the number of inputs per neurons. This number will determine the number of rows in `W1`.

The number of inputs to the hidden layer $3 \times 2$ which is **$6$**. Why? Because we are feeding three integers(i.e. characters) at a time and each of these integers are associated with a 2-D embedding vector. And the number of neurons in the hidden layer is a hyperparameter, meaning it is something the designer of the network has to choose. Andrej decided to go with $100$.

In [14]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100) #biases

Ideally, we would like to do something like this:

```py
embed @ W1 + b
```

Where we multiply the embedding matrix with the weights, add the biases and collect the results. But since the embeddings are stacked 📚🥞, the dimensions do not match. Consequently, the matrix multiplication cannot take place. To address this challenge, Andrej said we need to **concatenate** the embedding vectors. By concatenating those 2D vectors, we find ourselves with 6-D vectors that will be fed into the hidden layer. This is what we want. The remaining question is, how do you effeciently concatenate the embedding vectors. Andrej said there are many ways to do that in PyTorch because it is a large library.

Using the documentation, he showed us the [torch.cat](https://pytorch.org/docs/stable/generated/torch.cat.html#torch.cat) function which, according to the documentation, concatenates a sequence of tensors over a specified dimenstion. What Andrej did was he first _selected the embedding vector of the first character for each of the training example_, did the same thing for the second character, and the third character. By the end, he found himself with three tensors. The first with the embedding vectors of the first characters in the entire training set, the second containing the embedding vectors of the second characters in the entire training set, and the third containing the embedding vectors of the third characters in the training set. Each of the tensors is of the shape $32 \times 2$.

He finally concatenated the tensors along the dimension `1`. Which is along the rows. The first row of the three tensors are conctenanted together, the second row of the three tensors are concatenated together, up to the last row.

In Python, it looks like this:

In [15]:
torch.cat(
    [embed[:, 0, :], embed[:, 1, :], embed[:, 2, :]], 1
)

tensor([[-1.0041, -1.3760, -1.0041, -1.3760, -1.0041, -1.3760],
        [-1.0041, -1.3760, -1.0041, -1.3760, -1.8877, -0.5583],
        [-1.0041, -1.3760, -1.8877, -0.5583,  1.2296, -0.3806],
        [-1.8877, -0.5583,  1.2296, -0.3806,  1.2296, -0.3806],
        [ 1.2296, -0.3806,  1.2296, -0.3806,  0.1064,  0.3253],
        [-1.0041, -1.3760, -1.0041, -1.3760, -1.0041, -1.3760],
        [-1.0041, -1.3760, -1.0041, -1.3760,  1.2858,  0.7160],
        [-1.0041, -1.3760,  1.2858,  0.7160,  1.8109, -0.4273],
        [ 1.2858,  0.7160,  1.8109, -0.4273, -1.6619, -0.0768],
        [ 1.8109, -0.4273, -1.6619, -0.0768, -1.3248,  0.2001],
        [-1.6619, -0.0768, -1.3248,  0.2001, -1.6619, -0.0768],
        [-1.3248,  0.2001, -1.6619, -0.0768,  0.1064,  0.3253],
        [-1.0041, -1.3760, -1.0041, -1.3760, -1.0041, -1.3760],
        [-1.0041, -1.3760, -1.0041, -1.3760,  0.1064,  0.3253],
        [-1.0041, -1.3760,  0.1064,  0.3253, -1.3248,  0.2001],
        [ 0.1064,  0.3253, -1.3248,  0.2

Though we reached the point that we wanted, Andrej said that this is an ugly solution. Why because it will not generalize if we change the `block_size` which is now $3$. If later we want to increase it, we'd have to change the code because we are indexing directly. To improve the solution, Andrej showed the `torch.unbind` function which removes a specified dimension from a tensor.

In [16]:
torch.cat(torch.unbind(embed, 1), 1) #removes the first dimension, then concat along the first dimension

tensor([[-1.0041, -1.3760, -1.0041, -1.3760, -1.0041, -1.3760],
        [-1.0041, -1.3760, -1.0041, -1.3760, -1.8877, -0.5583],
        [-1.0041, -1.3760, -1.8877, -0.5583,  1.2296, -0.3806],
        [-1.8877, -0.5583,  1.2296, -0.3806,  1.2296, -0.3806],
        [ 1.2296, -0.3806,  1.2296, -0.3806,  0.1064,  0.3253],
        [-1.0041, -1.3760, -1.0041, -1.3760, -1.0041, -1.3760],
        [-1.0041, -1.3760, -1.0041, -1.3760,  1.2858,  0.7160],
        [-1.0041, -1.3760,  1.2858,  0.7160,  1.8109, -0.4273],
        [ 1.2858,  0.7160,  1.8109, -0.4273, -1.6619, -0.0768],
        [ 1.8109, -0.4273, -1.6619, -0.0768, -1.3248,  0.2001],
        [-1.6619, -0.0768, -1.3248,  0.2001, -1.6619, -0.0768],
        [-1.3248,  0.2001, -1.6619, -0.0768,  0.1064,  0.3253],
        [-1.0041, -1.3760, -1.0041, -1.3760, -1.0041, -1.3760],
        [-1.0041, -1.3760, -1.0041, -1.3760,  0.1064,  0.3253],
        [-1.0041, -1.3760,  0.1064,  0.3253, -1.3248,  0.2001],
        [ 0.1064,  0.3253, -1.3248,  0.2

Though the previous solution is better because we do not do direct indexing anymore... It turns out, there is an even better and more efficient solution which also gave the opportunity to hint at the internals of `torch.tensor`. To illustrate it, Andrej proposes us to create tensor of $18$ elements like so:

In [17]:
a = torch.arange(18)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

The shape of the previously created tensor is $18$. It turns out that we can quickly represent this very same tensor, as _different size `n-dimensional` tensors_. Though it is just a row-vector, we can "**view**" it anything else, and the way you do this is using the `view` function. Using this function, we can say "Oh, actually it is not single vector of $18$, but a $(2 \times 9)$ tensor, or even $(9 \times 2)$": 

In [18]:
a.view(2, 9) #turns the vector into a (2 x 9) tensor
#a.view(9, 2)
#a.view(3, 3, 2)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17]])

As long as multiplying the dimensions together results in the original size, it will work. And according to Andrej, the operation performed by `view` is very efficient. The reason is that _in each tensor, there is something called the underlying storage_. And the storage consist of the numbers in the tensors stored as a 1-D sequence. And this is how this vector is _represented in the computer memory_. It is always a 1-D sequence.

In [19]:
a.storage()

/var/folders/q0/sbqpw0b93pgf3f3qn_0c6g1m0000gn/T/ipykernel_19998/214256462.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a.storage()


 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

Call `view` function manipulates the attributes of the tensor that dictates how this 1-D sequence is interpreted (i.e. "viewed") as a tensor. The efficiency of `view` comes from the fact that _no memory is created, or copied_. It's just some of the internal attributes of tensor, namely the storage offset, the stride, and shape so that this 1-D sequence of bytes is "view" or "seen" as different n-dimensional array. Andrej recommended this [blog post, from Eric](http://blog.ezyang.com/2019/05/pytorch-internals/) to in further details and understanding how tensors are represented. He also said that he might create an entire video of this topic (torch internals) as well. But as of now, we keep in mind that the `view` function is an efficient function.

So to back to our `embed` matrix. We know the shape is $(32 \times 3 \times 2)$. $32$ because we are first considering the 32 training examples, but later will consider the entire. Using the `view` function, we can ask Pytorch to "view" our `embed` matrix, as a $32 \times 6$ instead. We can verify the following result by doing an element-wise equal check with of one of the previous results we got.

In [20]:
#embed.view(32, 6)
#torch.equal(torch.cat(torch.unbind(embed, 1), 1) , embed.view(32, 6)) #Element-wise check

So, to come back to our matrix multiplication that we wanted to use to compute the activation of the hidden layer of our MLP. We wanted to do something like this:

```py
embed @ W1 + b1
```

But the dimensions were not matching up. With what we learned, we can ask Pytorch to view it differently 😉

In [21]:
h = torch.tanh(embed.view(-1, 6) @ W1 + b1) #-1 lets PyTorch infer the right dimension

`h` contains the activations for all the training examples after being forward propagated through the 100 neurons in the hidden layer. Those values are $[-1, 1]$

Andrej brought our attention to the `+` operation that is happening as well because broadcasting happens there, so he walks us through how to make sure the broadcasting is doing what we want.

`b1.shape` is $100$. We have 

```py
32, 100
 1, 100
```
Broadcasting aligns on the right like above, and creates a fake dimension (1) on the left... turn everything into $(1 \times 100)$. Pythor will then copy vertically the $(1 \times 100)$ tensor $32$ times and do an element-wise addition.

It is correct because the same bias vector will be added to all of the rows containing the dot product of each training ex. with the neurons in the hidden layer. This is what we want.

# Implementing the output layer

Let's finally implement the output of layer of the neural network. This layer contains one neuron for each element of our vocabulary. In the case of the paper, the layer has $17,000$ neurons because it has $17,000$ possible words. In our case, we are a total of $27$ possible characters, so our layer will have $27$ neurons. 

In [22]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27) #biases

So, the logits (i.e. log-counts) which are the outputs of this of the neural network, will computed as follows:

In [23]:
logits = h @ W2 + b2

In [24]:
logits.shape

torch.Size([32, 27])

Just like in `part-1` we want to exponentiate those log-counts, to get what Andrej calld "fake-counts", and normalize them into a probability distribution.

In [25]:
counts = logits.exp()

In [26]:
prob = counts / counts.sum(1, keepdims=True)

# Implementing the negative log likelihood